In [ ]:
"""
Tic Tac Toe Player
"""
import copy
import math

X = "X"
O = "O"
EMPTY = None


def initial_state():
    """
    Returns starting state of the board.
    """
    return [[EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY]]

def number_of_xy(board):
    for i in board:
        for j in i:
            count_x=0
            count_y=0
            if j=='X':
                count_x+=1
            elif j=='Y':
                count_y+=1
    return count_x, count_y

def player(board):
    """
    Returns player who has the next turn on a board.
    """
    num_of_x, num_of_y=number_of_xy(board)
    if num_of_x==num_of_y and num_of_y==0:
        return 'X'
    if num_of_x>=num_of_y:
        return 'Y'
    else:
        return 'X'

def actions(board):
    """
    Returns set of all possible actions (i, j) available on the board.
    """
    actions=set()
    for i in range(3):
        for j in range(3):
            if board[i][j]=='EMPTY':
                actions.add((i,j))

    return actions

def result(board, action):
    """
    Returns the board that results from making move (i, j) on the board.
    """
    if board[action[0]][action[1]]!='EMPTY':
        raise Exception
    else:
        new_board=copy.deepcopy(board)
        next_player=player(board)
        new_board[action[0]][action[1]]=next_player
        return new_board


def winner(board):
    """
    Returns the winner of the game, if there is one.
    """
    # check for x
    # check for horizontal row
    for i in board:
        count=0
        for j in i:
            if j=='X':
                count+=1
        if count==3:
            return 'X'
    # check for column
    for i in range(3):
        count=0
        for j in range(3):
            if board[j][i]=='X':
                count+=1
        if count==3:
            return 'X'
    # check for diagonal 1
    count = 0
    for i in range(3):
        if board[i][i]=='X':
            count+=1
    if count==3:
        return 'X'
    # check for diagonal 2
    if board[0][2]=='X' and board[1][1]=='X' and board[2][0]=='X':
        return 'X'

        # check for x
        # check for horizontal row
    for i in board:
        count = 0
        for j in i:
            if j == 'Y':
                count += 1
        if count == 3:
            return 'Y'
        # check for column
    for i in range(3):
        count = 0
        for j in range(3):
            if board[j][i] == 'Y':
                count += 1
        if count == 3:
            return 'Y'
        # check for diagonal 1
    count = 0
    for i in range(3):
        if board[i][i] == 'Y':
            count += 1
    if count == 3:
        return 'Y'
    # check for diagonal 2
    if board[0][2] == 'Y' and board[1][1] == 'Y' and board[2][0] == 'Y':
        return 'Y'

    return None
def terminal(board):
    """
    Returns True if game is over, False otherwise.
    """
    if winner(board)!=None or len(actions(board))==0:
        return True
    else:
        return False


def utility(board):
    """
    Returns 1 if X has won the game, -1 if O has won, 0 otherwise.
    """
    if(winner(board)=='X'):
        return 1
    elif(winner(board)=='Y'):
        return -1
    else:
        return 0

def cumulative_rew(board, player, sum_rew):
    if terminal(board):
        return sum_rew+utility(board)
    else:
        actions_new=actions(board)
        if player=='X':
            return max(map(lambda y: cumulative_rew(y,'Y',sum_rew),map(lambda x: result(board,x),actions_new)))
        if player=='Y':
            return min(map(lambda y: cumulative_rew(y, 'X', sum_rew), map(lambda x: result(board, x), actions_new)))


def minimax(board):
    """
    Returns the optimal action for the current player on the board.
    """
    if terminal(board):
        return None

    action_space=actions(board)
    current_player=player(board)

    if current_player=='X':
        maximum=(-math.inf,None)
        for i in action_space:
            new=result(board,i)
            reward=cumulative_rew(new,'X',0)
            if reward>maximum[0]:
                maximum=(reward,i)
        return maximum[1]
    else:
        minimum = (math.inf, None)
        for i in action_space:
            new = result(board, i)
            reward = cumulative_rew(new, 'X', 0)
            if reward < minimum[0]:
                minimum = (reward, i)
        return minimum[1]






In [ ]:
import csv
import itertools
import sys

PROBS = {

    # Unconditional probabilities for having gene
    "gene": {
        2: 0.01,
        1: 0.03,
        0: 0.96
    },

    "trait": {

        # Probability of trait given two copies of gene
        2: {
            True: 0.65,
            False: 0.35
        },

        # Probability of trait given one copy of gene
        1: {
            True: 0.56,
            False: 0.44
        },

        # Probability of trait given no gene
        0: {
            True: 0.01,
            False: 0.99
        }
    },

    # Mutation probability
    "mutation": 0.01
}


def main():

    # Check for proper usage
    if len(sys.argv) != 2:
        sys.exit("Usage: python h.py data.csv")
    people = load_data(sys.argv[1])

    # Keep track of gene and trait probabilities for each person
    probabilities = {
        person: {
            "gene": {
                2: 0,
                1: 0,
                0: 0
            },
            "trait": {
                True: 0,
                False: 0
            }
        }
        for person in people
    }

    # Loop over all sets of people who might have the trait
    names = set(people)
    for have_trait in powerset(names):

        # Check if current set of people violates known information
        fails_evidence = any(
            (people[person]["trait"] is not None and
             people[person]["trait"] != (person in have_trait))
            for person in names
        )
        if fails_evidence:
            continue

        # Loop over all sets of people who might have the gene
        for one_gene in powerset(names):
            for two_genes in powerset(names - one_gene):

                # Update probabilities with new joint probability
                p = joint_probability(people, one_gene, two_genes, have_trait)
                update(probabilities, one_gene, two_genes, have_trait, p)

    # Ensure probabilities sum to 1
    normalize(probabilities)

    # Print results
    for person in people:
        print(f"{person}:")
        for field in probabilities[person]:
            print(f"  {field.capitalize()}:")
            for value in probabilities[person][field]:
                p = probabilities[person][field][value]
                print(f"    {value}: {p:.4f}")


def load_data(filename):
    """
    Load gene and trait data from a file into a dictionary.
    File assumed to be a CSV containing fields name, mother, father, trait.
    mother, father must both be blank, or both be valid names in the CSV.
    trait should be 0 or 1 if trait is known, blank otherwise.
    """
    data = dict()
    with open(filename) as f:
        reader = csv.DictReader(f)
        for row in reader:
            name = row["name"]
            data[name] = {
                "name": name,
                "mother": row["mother"] or None,
                "father": row["father"] or None,
                "trait": (True if row["trait"] == "1" else
                          False if row["trait"] == "0" else None)
            }
    return data


def powerset(s):
    """
    Return a list of all possible subsets of set s.
    """
    s = list(s)
    return [
        set(s) for s in itertools.chain.from_iterable(
            itertools.combinations(s, r) for r in range(len(s) + 1)
        )
    ]

def case_1(person,one_gene,two_genes,have_trait):
    if person in one_gene:
        if person in have_trait:
            return PROBS['gene'][1]*PROBS['trait'][1][True]
        else:
            return PROBS['gene'][1] * PROBS['trait'][1][False]
    elif person in two_genes:
        if person in have_trait:
            return PROBS['gene'][2]*PROBS['trait'][1][True]
        else:
            return PROBS['gene'][2] * PROBS['trait'][1][False]
    else:
        if person in have_trait:
            return PROBS['gene'][0]*PROBS['trait'][1][True]
        else:
            return PROBS['gene'][0] * PROBS['trait'][1][False]

def find_parent_genes(person,one_gene,two_genes):
    person_m = person['mother']
    person_f = person['father']
    genes=[]
    if person_m in one_gene:
        genes.append(1)
    elif person_m in two_genes:
        genes.append(2)
    else:
        genes.append(0)
    if person_f in one_gene:
        genes.append(1)
    elif person_f in two_genes:
        genes.append(2)
    else:
        genes.append(0)

    return genes

def get_probs_from_parent(genes,n):
    m=PROBS['mutation']
    if 1 not in genes and 2 not in genes:#[0,0]
        if n==0:
            return (1-m)*(1-m)
        if n==1:
            return ((1-m)*m)*2
        if n==2:
            return (m*m)

    if 1 in genes and 0 in genes and 2 not in genes:  # [0,1]
        if n == 0:
            return ((1-m)*m*0.5)+((1-m)*(1-m)*0.5)+(m*0.5*(1-m))+(m*0.5*m)
        if n == 1:
            return (m*m*0.5)+(m*(1-m)*0.5)+((1-m)*(1-m)*0.5)+((1-m)*m*0.5)
        if n == 2:
            return (m*(1-m)*0.5)+(m*m*0.5)

    if 1 in genes and 0 not in genes and 2 not in genes:  # [1,1]
        if n == 0:
            return 0.25*(((1-m)*(1-m))+((1-m)*(m))*2+((m)*(m)))
        if n == 1:
            return 0.25*((m*m)+(m*(1-m))+((1-m)*(1-m))*2)
        if n == 2:
            return 0.25*((m*m)+(m*(1-m))*2+((1-m)*(1-m)))

    if 1 in genes and 0 not in genes and 2 in genes:  # [1,2]
        if n == 0:
            return 0.25*(2*(m*(1-m))+2*(m*m))
        if n == 1:
            return 0.25*(2*((1-m)*(1-m))+4*(m*(1-m)))
        if n == 2:
            return 0.25*(2*(m*(1-m))+2*((1-m)*(1-m)))

    if 1 not in genes and 0 not in genes and 2 in genes:  # [2,2]
        if n == 0:
            return m*m
        if n == 1:
            return 2*(m*(1-m))
        if n == 2:
            return (1-m)*(1-m)

    if 1 not in genes and 0 not in genes and 2 in genes:  # [0,2]
        if n == 0:
            return m*(1-m)
        if n == 1:
            return m*(1-m)
        if n == 2:
            return (m)*(1-m)
def case_2(p,one_gene,two_genes,have_trait):
    person=p['name']
    genes=find_parent_genes(p,one_gene,two_genes)
    p1=None
    p2=None

    if person in one_gene:
        p2=get_probs_from_parent(genes,1)
        if person in have_trait:
            p1=PROBS['trait'][1][True]
        else:
            p1=PROBS['trait'][1][False]
        return p2*p1
    elif person in two_genes:
        p2 = get_probs_from_parent(genes, 2)
        if person in have_trait:
            p1=PROBS['trait'][1][True]
        else:
            p1=PROBS['trait'][1][False]
        return p2 * p1
    else:
        p2 = get_probs_from_parent(genes, 0)
        if person in have_trait:
            p1=PROBS['trait'][1][True]
        else:
            p1=PROBS['trait'][1][False]
        return p2 * p1

def joint_probability(people, one_gene, two_genes, have_trait):
    """
    Compute and return a joint probability.

    The probability returned should be the probability that
        * everyone in set `one_gene` has one copy of the gene, and
        * everyone in set `two_genes` has two copies of the gene, and
        * everyone not in `one_gene` or `two_gene` does not have the gene, and
        * everyone in set `have_trait` has the trait, and
        * everyone not in set` have_trait` does not have the trait.
    """
    p=1
    for i in people:
        person_m=i['mother']
        person_f=i['father']
        if person_m==None or person_f==None:
            p*=case_1(i['name'],one_gene, two_genes, have_trait)
        else:
            p*case_2(i,one_gene, two_genes, have_trait)
    return p

def update(probabilities, one_gene, two_genes, have_trait, p):
    """
    Add to `probabilities` a new joint probability `p`.
    Each person should have their "gene" and "trait" distributions updated.
    Which value for each distribution is updated depends on whether
    the person is in `have_gene` and `have_trait`, respectively.
    """
    for i in probabilities:
        if i in one_gene:
            probabilities[i]['gene'][1]=p
        elif i in two_genes:
            probabilities[i]['gene'][2]=p
        else:
            probabilities[i]['gene'][0]=p

        if i in have_trait:
            probabilities[i]['trait'][True]=p
        else:
            probabilities[i]['trait'][False]=p

def normalize(probabilities):
    """
    Update `probabilities` such that each probability distribution
    is normalized (i.e., sums to 1, with relative proportions the same).
    """
    s=0
    if sum(probabilities.values())!=1:
        s=sum(probabilities.values())

    for i in probabilities.keys():
        probabilities[i]/=s


if __name__ == "__main__":
    main()


# degrees- degrees.py


In [ ]:
import csv
import sys

from util import Node, StackFrontier, QueueFrontier

# Maps names to a set of corresponding person_ids
names = {}

# Maps person_ids to a dictionary of: name, birth, movies (a set of movie_ids)
people = {}

# Maps movie_ids to a dictionary of: title, year, stars (a set of person_ids)
movies = {}


def load_data(directory):
    """
    Load data from CSV files into memory.
    """
    # Load people
    with open(f"{directory}/people.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            people[row["id"]] = {
                "name": row["name"],
                "birth": row["birth"],
                "movies": set()
            }
            if row["name"].lower() not in names:
                names[row["name"].lower()] = {row["id"]}
            else:
                names[row["name"].lower()].add(row["id"])

    # Load movies
    with open(f"{directory}/movies.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            movies[row["id"]] = {
                "title": row["title"],
                "year": row["year"],
                "stars": set()
            }

    # Load stars
    with open(f"{directory}/stars.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            try:
                people[row["person_id"]]["movies"].add(row["movie_id"])
                movies[row["movie_id"]]["stars"].add(row["person_id"])
            except KeyError:
                pass


def main():
    if len(sys.argv) > 2:
        sys.exit("Usage: python degrees.py [directory]")
    directory = sys.argv[1] if len(sys.argv) == 2 else "small"

    # Load data from files into memory
    print("Loading data...")
    load_data(directory)
    print("Data loaded.")

    source = person_id_for_name(input("Name: "))
    if source is None:
        sys.exit("Person not found.")
    target = person_id_for_name(input("Name: "))
    if target is None:
        sys.exit("Person not found.")

    path = shortest_path(source, target)

    if path is None:
        print("Not connected.")
    else:
        degrees = len(path)
        print(f"{degrees} degrees of separation.")
        path = [(None, source)] + path
        for i in range(degrees):
            person1 = people[path[i][1]]["name"]
            person2 = people[path[i + 1][1]]["name"]
            movie = movies[path[i + 1][0]]["title"]
            print(f"{i + 1}: {person1} and {person2} starred in {movie}")


def shortest_path(source, target):
    """
    Returns the shortest list of (movie_id, person_id) pairs
    that connect the source to the target.

    If no possible path, returns None.
    """

    src_node=(None,source)
    path=[]
    explored=[]
    front=QueueFrontier()
    front.add(src_node)

    while(src_node[1]!=target and not front.empty()):
        src_node = front.remove()
        if src_node in explored:
            continue
        neighbours=neighbors_for_person(src_node[1])
        explored.append(src_node)
        for i in neighbours:
            front.add(i)
        path.append(src_node)

    if src_node[1]==target:
        return path[1:]
    else:
        return None




def person_id_for_name(name):
    """
    Returns the IMDB id for a person's name,
    resolving ambiguities as needed.
    """
    person_ids = list(names.get(name.lower(), set()))
    if len(person_ids) == 0:
        return None
    elif len(person_ids) > 1:
        print(f"Which '{name}'?")
        for person_id in person_ids:
            person = people[person_id]
            name = person["name"]
            birth = person["birth"]
            print(f"ID: {person_id}, Name: {name}, Birth: {birth}")
        try:
            person_id = input("Intended Person ID: ")
            if person_id in person_ids:
                return person_id
        except ValueError:
            pass
        return None
    else:
        return person_ids[0]


def neighbors_for_person(person_id):
    """
    Returns (movie_id, person_id) pairs for people
    who starred with a given person.
    """
    movie_ids = people[person_id]["movies"]
    neighbors = set()
    for movie_id in movie_ids:
        for person_id in movies[movie_id]["stars"]:
            neighbors.add((movie_id, person_id))
    return neighbors


if __name__ == "__main__":
    main()


# ttt- ttt.py


In [ ]:
import pygame
import sys
import time

import ttt as ttt

pygame.init()
size = width, height = 600, 400

# Colors
black = (0, 0, 0)
white = (255, 255, 255)

screen = pygame.display.set_mode(size)

mediumFont = pygame.font.Font("OpenSans-Regular.ttf", 28)
largeFont = pygame.font.Font("OpenSans-Regular.ttf", 40)
moveFont = pygame.font.Font("OpenSans-Regular.ttf", 60)

user = None
board = ttt.initial_state()
ai_turn = False

while True:

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            sys.exit()

    screen.fill(black)

    # Let user choose a player.
    if user is None:

        # Draw title
        title = largeFont.render("Play Tic-Tac-Toe", True, white)
        titleRect = title.get_rect()
        titleRect.center = ((width / 2), 50)
        screen.blit(title, titleRect)

        # Draw buttons
        playXButton = pygame.Rect((width / 8), (height / 2), width / 4, 50)
        playX = mediumFont.render("Play as X", True, black)
        playXRect = playX.get_rect()
        playXRect.center = playXButton.center
        pygame.draw.rect(screen, white, playXButton)
        screen.blit(playX, playXRect)

        playOButton = pygame.Rect(5 * (width / 8), (height / 2), width / 4, 50)
        playO = mediumFont.render("Play as O", True, black)
        playORect = playO.get_rect()
        playORect.center = playOButton.center
        pygame.draw.rect(screen, white, playOButton)
        screen.blit(playO, playORect)

        # Check if button is clicked
        click, _, _ = pygame.mouse.get_pressed()
        if click == 1:
            mouse = pygame.mouse.get_pos()
            if playXButton.collidepoint(mouse):
                time.sleep(0.2)
                user = ttt.X
            elif playOButton.collidepoint(mouse):
                time.sleep(0.2)
                user = ttt.O

    else:

        # Draw game board
        tile_size = 80
        tile_origin = (width / 2 - (1.5 * tile_size),
                       height / 2 - (1.5 * tile_size))
        tiles = []
        for i in range(3):
            row = []
            for j in range(3):
                rect = pygame.Rect(
                    tile_origin[0] + j * tile_size,
                    tile_origin[1] + i * tile_size,
                    tile_size, tile_size
                )
                pygame.draw.rect(screen, white, rect, 3)

                if board[i][j] != ttt.EMPTY:
                    move = moveFont.render(board[i][j], True, white)
                    moveRect = move.get_rect()
                    moveRect.center = rect.center
                    screen.blit(move, moveRect)
                row.append(rect)
            tiles.append(row)

        game_over = ttt.terminal(board)
        player = ttt.player(board)

        # Show title
        if game_over:
            winner = ttt.winner(board)
            if winner is None:
                title = f"Game Over: Tie."
            else:
                title = f"Game Over: {winner} wins."
        elif user == player:
            title = f"Play as {user}"
        else:
            title = f"Computer thinking..."
        title = largeFont.render(title, True, white)
        titleRect = title.get_rect()
        titleRect.center = ((width / 2), 30)
        screen.blit(title, titleRect)

        # Check for AI move
        if user != player and not game_over:
            if ai_turn:
                time.sleep(0.5)
                move = ttt.minimax(board)
                board = ttt.result(board, move)
                ai_turn = False
            else:
                ai_turn = True

        # Check for a user move
        click, _, _ = pygame.mouse.get_pressed()
        if click == 1 and user == player and not game_over:
            mouse = pygame.mouse.get_pos()
            for i in range(3):
                for j in range(3):
                    if (board[i][j] == ttt.EMPTY and tiles[i][j].collidepoint(mouse)):
                        board = ttt.result(board, (i, j))

        if game_over:
            againButton = pygame.Rect(width / 3, height - 65, width / 3, 50)
            again = mediumFont.render("Play Again", True, black)
            againRect = again.get_rect()
            againRect.center = againButton.center
            pygame.draw.rect(screen, white, againButton)
            screen.blit(again, againRect)
            click, _, _ = pygame.mouse.get_pressed()
            if click == 1:
                mouse = pygame.mouse.get_pos()
                if againButton.collidepoint(mouse):
                    time.sleep(0.2)
                    user = None
                    board = ttt.initial_state()
                    ai_turn = False

    pygame.display.flip()


# knights- puzzle.py

In [ ]:
from logic import *

AKnight = Symbol("A is a Knight")
AKnave = Symbol("A is a Knave")

BKnight = Symbol("B is a Knight")
BKnave = Symbol("B is a Knave")

CKnight = Symbol("C is a Knight")
CKnave = Symbol("C is a Knave")

# Puzzle 0
# A says "I am both a knight and a knave."
knowledge0 = And(
    Or(AKnave,AKnight),
    Implication(AKnight, Not(AKnave)),
    Implication(AKnave, Not(AKnight)),
    Implication(AKnight,And(AKnight,AKnave)),
    Implication(AKnave,Not(And(AKnight,AKnave)))
)

# Puzzle 1
# A says "We are both knaves."
# B says nothing.
knowledge1 = And(
    Or(AKnave,AKnight),
    Or(BKnave,BKnight),
    Implication(AKnight, Not(AKnave)),
    Implication(AKnave, Not(AKnight)),
    Implication(BKnight, Not(BKnave)),
    Implication(BKnave, Not(BKnight)),
    Implication(AKnight,And(AKnave,BKnave)),
    Implication(AKnave,Not(And(AKnave,BKnave)))
)

# Puzzle 2
# A says "We are the same kind."
# B says "We are of different kinds."
knowledge2 = And(
    Or(AKnave,AKnight),
    Or(BKnave,BKnight),
    Implication(AKnight, Not(AKnave)),
    Implication(AKnave, Not(AKnight)),
    Implication(BKnight, Not(BKnave)),
    Implication(BKnave, Not(BKnight)),
    Implication(AKnight,Or(And(AKnave,BKnave),And(AKnight,BKnight))),
    Implication(AKnave,Not(Or(And(AKnave,BKnave),And(AKnight,BKnight)))),
    Implication(BKnight,Or(And(AKnave,BKnight),And(AKnight,BKnave))),
    Implication(BKnave,Not(Or(And(AKnave,BKnight),And(AKnight,BKnave)))),
)

# Puzzle 3
# A says either "I am a knight." or "I am a knave.", but you don't know which.
# B says "A said 'I am a knave'."
# B says "C is a knave."
# C says "A is a knight."
knowledge3 = And(
    Or(AKnave,AKnight),
    Or(BKnave,BKnight),
    Or(CKnave,CKnight),
    Implication(AKnight, Not(AKnave)),
    Implication(AKnave, Not(AKnight)),
    Implication(BKnight, Not(BKnave)),
    Implication(BKnave, Not(AKnight)),
    Implication(CKnight, Not(CKnave)),
    Implication(CKnave, Not(CKnight)),
    Implication(AKnight,Or(AKnight,AKnave)),
    Implication(AKnave,Or(AKnight,AKnave)),
    Implication(BKnight,And(Or(Implication(AKnight,BKnave),Implication(AKnave,Not(BKnave))),CKnave)),
    Implication(BKnave,Not(And(Or(Implication(AKnight,BKnave),Implication(AKnave,Not(BKnave))),CKnave))),
    Implication(CKnight,AKnight),
    Implication(CKnave,Not(AKnight))
)


def main():
    symbols = [AKnight, AKnave, BKnight, BKnave, CKnight, CKnave]
    puzzles = [
        ("Puzzle 0", knowledge0),
        ("Puzzle 1", knowledge1),
        ("Puzzle 2", knowledge2),
        ("Puzzle 3", knowledge3)
    ]
    for puzzle, knowledge in puzzles:
        print(puzzle)
        if len(knowledge.conjuncts) == 0:
            print("    Not yet implemented.")
        else:
            for symbol in symbols:
                if model_check(knowledge, symbol):
                    print(f"    {symbol}")


if __name__ == "__main__":
    main()


# minesweeper.py

In [ ]:
import itertools
import random


class Minesweeper():
    """
    Minesweeper game representation
    """

    def __init__(self, height=8, width=8, mines=8):

        # Set initial width, height, and number of mines
        self.height = height
        self.width = width
        self.mines = set()

        # Initialize an empty field with no mines
        self.board = []
        for i in range(self.height):
            row = []
            for j in range(self.width):
                row.append(False)
            self.board.append(row)

        # Add mines randomly
        while len(self.mines) != mines:
            i = random.randrange(height)
            j = random.randrange(width)
            if not self.board[i][j]:
                self.mines.add((i, j))
                self.board[i][j] = True

        # At first, player has found no mines
        self.mines_found = set()

    def print(self):
        """
        Prints a text-based representation
        of where mines are located.
        """
        for i in range(self.height):
            print("--" * self.width + "-")
            for j in range(self.width):
                if self.board[i][j]:
                    print("|X", end="")
                else:
                    print("| ", end="")
            print("|")
        print("--" * self.width + "-")

    def is_mine(self, cell):
        i, j = cell
        return self.board[i][j]

    def nearby_mines(self, cell):
        """
        Returns the number of mines that are
        within one row and column of a given cell,
        not including the cell itself.
        """

        # Keep count of nearby mines
        count = 0

        # Loop over all cells within one row and column
        for i in range(cell[0] - 1, cell[0] + 2):
            for j in range(cell[1] - 1, cell[1] + 2):

                # Ignore the cell itself
                if (i, j) == cell:
                    continue

                # Update count if cell in bounds and is mine
                if 0 <= i < self.height and 0 <= j < self.width:
                    if self.board[i][j]:
                        count += 1

        return count

    def won(self):
        """
        Checks if all mines have been flagged.
        """
        return self.mines_found == self.mines


class Sentence():
    """
    Logical statement about a Minesweeper game
    A sentence consists of a set of board cells,
    and a count of the number of those cells which are mines.
    """

    def __init__(self, cells, count):
        self.cells = set(cells)
        self.count = count

    def __eq__(self, other):
        return self.cells == other.cells and self.count == other.count

    def __str__(self):
        return f"{self.cells} = {self.count}"

    def known_mines(self):
        """
        Returns the set of all cells in self.cells known to be mines.
        """
        if self.count==len(self.cells) and self.count!=0:
            return self.cells
        return set()

    def known_safes(self):
        """
        Returns the set of all cells in self.cells known to be safe.
        """
        if self.count==0:
            return self.cells
        return set()

    def mark_mine(self, cell):
        """
        Updates internal knowledge representation given the fact that
        a cell is known to be a mine.
        """
        if cell  not in self.cells:
            return
        else:
            self.cells.remove(cell)
            self.count-=1


    def mark_safe(self, cell):
        """
        Updates internal knowledge representation given the fact that
        a cell is known to be safe.
        """
        if cell  not in self.cells:
            return
        else:
            self.cells.remove(cell)

class MinesweeperAI():
    """
    Minesweeper game player
    """

    def __init__(self, height=8, width=8):

        # Set initial height and width
        self.height = height
        self.width = width

        # Keep track of which cells have been clicked on
        self.moves_made = set()

        # Keep track of cells known to be safe or mines
        self.mines = set()
        self.safes = set()

        # List of sentences about the game known to be true
        self.knowledge = []

    def mark_mine(self, cell):
        """
        Marks a cell as a mine, and updates all knowledge
        to mark that cell as a mine as well.
        """
        self.mines.add(cell)
        for sentence in self.knowledge:
            sentence.mark_mine(cell)

    def mark_safe(self, cell):
        """
        Marks a cell as safe, and updates all knowledge
        to mark that cell as safe as well.
        """
        self.safes.add(cell)
        for sentence in self.knowledge:
            sentence.mark_safe(cell)

    def find_neighbours(self,cell):
        possible_neighbours=set()
        row=cell[0]
        col=cell[1]
        possible_neighbours.add((row,col-1))
        possible_neighbours.add((row, col + 1))
        possible_neighbours.add((row-1, col - 1))
        possible_neighbours.add((row - 1, col +1))
        possible_neighbours.add((row - 1, col))
        possible_neighbours.add((row - 1, col - 1))
        possible_neighbours.add((row + 1, col + 1))
        possible_neighbours.add((row + 1, col))
        possible_neighbours.add((row + 1, col - 1))

        neighbours=set()
        for i in possible_neighbours:
            if(row in range(self.height) and col in range(self.width)):
                neighbours.add(i)

        return neighbours

    def check_kb_sentences(self):
        changes=0
        for s in self.knowledge:
            mines_per_sentance=s.known_mines()
            safes_per_sentance=s.known_safes()
            for i in mines_per_sentance:
                self.mark_mine(i)
            for i in safes_per_sentance:
                self.mark_safe(i)
            if mines_per_sentance or safes_per_sentance:
                changes+=1
        if changes!=0:
            self.infer()


    def infer(self):
        changes=0
        self.check_kb_sentences()
        for i in range(len(self.knowledge)):
            for j in range(len(self.knowledge)-i-1):
                sent1=self.knowledge[i]
                sent2=self.knowledge[j]
                obj1_cells=sent1.cells
                obj2_cells=sent2.cells
                obj1_count=sent1.count
                obj2_count=sent2.count
                if obj1_cells.issubset(obj2_cells):
                    changes+=1
                    new_set=obj2_cells.differece(obj1_cells)
                    new_count=obj2_count-obj1_count
                    self.knowledge.remove(self.knowledge[j])
                    new_sent=Sentence(new_set,new_count)
                    self.knowledge.append(new_sent)
                elif obj2_cells.issubset(obj1_cells):
                    changes+=1
                    new_set=obj1_cells.differece(obj2_cells)
                    new_count=obj1_count-obj2_count
                    self.knowledge.remove(self.knowledge[i])
                    new_sent=Sentence(new_set,new_count)
                    self.knowledge.append(new_sent)
        if changes!=0:
            self.infer()








    def add_knowledge(self, cell, count):
        """
        Called when the Minesweeper board tells us, for a given
        safe cell, how many neighboring cells have mines in them.

        This function should:
            1) mark the cell as a move that has been made
            2) mark the cell as safe
            3) add a new sentence to the AI's knowledge base
               based on the value of `cell` and `count`
            4) mark any additional cells as safe or as mines
               if it can be concluded based on the AI's knowledge base
            5) add any new sentences to the AI's knowledge base
               if they can be inferred from existing knowledge
        """
        self.moves_made.add(cell)
        self.safes.add(cell)
        cell_neighbours=self.find_neighbours(cell)
        new_sentence=Sentence(cell_neighbours,count)
        self.knowledge.append(new_sentence)
        self.infer()


    def make_safe_move(self):
        """
        Returns a safe cell to choose on the Minesweeper board.
        The move must be known to be safe, and not already a move
        that has been made.

        This function may use the knowledge in self.mines, self.safes
        and self.moves_made, but should not modify any of those values.
        """
        for i in self.safes:
            if i not in self.moves_made:
                return i
        return None

    def make_random_move(self):
        """
        Returns a move to make on the Minesweeper board.
        Should choose randomly among cells that:
            1) have not already been chosen, and
            2) are not known to be mines
        """
        random_array=[]
        for row in range(self.height):
            for col in range(self.width):
                cell=(row,col)
                if cell not in self.moves_made and cell not in self.mines:
                    random_array.append(cell)
        return random.choices(random_array,k=1)


# pr

In [ ]:
import copy
import os
import random
import re
import sys

DAMPING = 0.85
SAMPLES = 10000


def main():
    if len(sys.argv) != 2:
        sys.exit("Usage: python pagerank.py corpus")
    corpus = crawl(sys.argv[1])
    ranks = sample_pagerank(corpus, DAMPING, SAMPLES)
    print(f"PageRank Results from Sampling (n = {SAMPLES})")
    for page in sorted(ranks):
        print(f"  {page}: {ranks[page]:.4f}")
    ranks = iterate_pagerank(corpus, DAMPING)
    print(f"PageRank Results from Iteration")
    for page in sorted(ranks):
        print(f"  {page}: {ranks[page]:.4f}")


def crawl(directory):
    """
    Parse a directory of HTML pages and check for links to other pages.
    Return a dictionary where each key is a page, and values are
    a list of all other pages in the corpus that are linked to by the page.
    """
    pages = dict()

    # Extract all links from HTML files
    for filename in os.listdir(directory):
        if not filename.endswith(".html"):
            continue
        with open(os.path.join(directory, filename)) as f:
            contents = f.read()
            links = re.findall(r"<a\s+(?:[^>]*?)href=\"([^\"]*)\"", contents)
            pages[filename] = set(links) - {filename}

    # Only include links to other pages in the corpus
    for filename in pages:
        pages[filename] = set(
            link for link in pages[filename]
            if link in pages
        )

    return pages


def transition_model(corpus, page, damping_factor):
    """
    Return a probability distribution over which page to visit next,
    given a current page.

    With probability `damping_factor`, choose a link at random
    linked to by `page`. With probability `1 - damping_factor`, choose
    a link at random chosen from all pages in the corpus.
    """

    new_dicto=dict()

    linked_pages=corpus[page]
    linked_pages_count=len(linked_pages)
    all_pages=list(corpus.keys())
    all_pages_count=len(all_pages)
    d=damping_factor
    d_bar=1-damping_factor

    for i in linked_pages:
        new_dicto[i]=d/linked_pages_count
    for j in all_pages:
        if j in list(new_dicto.keys()):
            new_dicto[j]+=d_bar/all_pages_count
        else:
            new_dicto=d_bar/all_pages_count

    return new_dicto


def sample_pagerank(corpus, damping_factor, n):
    """
    Return PageRank values for each page by sampling `n` pages
    according to transition model, starting with a page at random.

    Return a dictionary where keys are page names, and values are
    their estimated PageRank value (a value between 0 and 1). All
    PageRank values should sum to 1.
    """
    tm=None
    page_hits=dict()
    for i in list(corpus.keys()):
        page_hits[i]=0

    page=None
    for i in range(1,n):
        if page==None:
            page=random.choices(list(corpus.keys()),k=1)[0]
        tm=transition_model(corpus,page,damping_factor)
        page_hits[page]=((i-1)*page_hits[page]+tm[page])/i
        page_links=corpus[page]
        page=random.choices(page_links,weights=page_hits,k=1)[0]

    return page_hits

def find_pr(corpus, damping_factor, p,ranks,threshold=0.001):
    page=copy.deepcopy(p)
    num_of_pages = len(corpus.keys())
    old=copy.deepcopy(ranks)
    while (1):
        linked_pages = corpus[page]
        if linked_pages == set():
            linked_pages = list(corpus.keys())
            new_page = random.choices(linked_pages, k=1)[0]
        else:
            new_page = random.choices(linked_pages, k=1)[0]

        summation=0
        for i in linked_pages:
            ranks=find_pr(corpus,damping_factor,i,ranks)
            summation+=ranks[i]/len(corpus[i])
        pr = ((1 - damping_factor) / num_of_pages) + damping_factor(summation)
        ranks[page]=pr
        page=new_page
        count=0
        for i in range(len(old)):
            if abs(old[i]-ranks[i])<0.001:
                count+=1
        if(count==num_of_pages):
            break
    return ranks

def iterate_pagerank(corpus, damping_factor):
    """
    Return PageRank values for each page by iteratively updating
    PageRank values until convergence.

    Return a dictionary where keys are page names, and values are
    their estimated PageRank value (a value between 0 and 1). All
    PageRank values should sum to 1.
    """
    ranks=dict()
    num_of_pages=len(corpus.keys())
    for i in list(corpus.keys()):
        ranks[i]=1/num_of_pages

    all_pages=list(corpus.keys())
    page=random.choices(all_pages,k=1)[0]
    ranks=find_pr(corpus,damping_factor,page,ranks)

    return ranks

if __name__ == "__main__":
    main()


# heredity.py

In [ ]:
import csv
import itertools
import sys

PROBS = {

    # Unconditional probabilities for having gene
    "gene": {
        2: 0.01,
        1: 0.03,
        0: 0.96
    },

    "trait": {

        # Probability of trait given two copies of gene
        2: {
            True: 0.65,
            False: 0.35
        },

        # Probability of trait given one copy of gene
        1: {
            True: 0.56,
            False: 0.44
        },

        # Probability of trait given no gene
        0: {
            True: 0.01,
            False: 0.99
        }
    },

    # Mutation probability
    "mutation": 0.01
}


def main():

    # Check for proper usage
    if len(sys.argv) != 2:
        sys.exit("Usage: python h.py data.csv")
    people = load_data(sys.argv[1])

    # Keep track of gene and trait probabilities for each person
    probabilities = {
        person: {
            "gene": {
                2: 0,
                1: 0,
                0: 0
            },
            "trait": {
                True: 0,
                False: 0
            }
        }
        for person in people
    }

    # Loop over all sets of people who might have the trait
    names = set(people)
    for have_trait in powerset(names):

        # Check if current set of people violates known information
        fails_evidence = any(
            (people[person]["trait"] is not None and
             people[person]["trait"] != (person in have_trait))
            for person in names
        )
        if fails_evidence:
            continue

        # Loop over all sets of people who might have the gene
        for one_gene in powerset(names):
            for two_genes in powerset(names - one_gene):

                # Update probabilities with new joint probability
                p = joint_probability(people, one_gene, two_genes, have_trait)
                update(probabilities, one_gene, two_genes, have_trait, p)

    # Ensure probabilities sum to 1
    normalize(probabilities)

    # Print results
    for person in people:
        print(f"{person}:")
        for field in probabilities[person]:
            print(f"  {field.capitalize()}:")
            for value in probabilities[person][field]:
                p = probabilities[person][field][value]
                print(f"    {value}: {p:.4f}")


def load_data(filename):
    """
    Load gene and trait data from a file into a dictionary.
    File assumed to be a CSV containing fields name, mother, father, trait.
    mother, father must both be blank, or both be valid names in the CSV.
    trait should be 0 or 1 if trait is known, blank otherwise.
    """
    data = dict()
    with open(filename) as f:
        reader = csv.DictReader(f)
        for row in reader:
            name = row["name"]
            data[name] = {
                "name": name,
                "mother": row["mother"] or None,
                "father": row["father"] or None,
                "trait": (True if row["trait"] == "1" else
                          False if row["trait"] == "0" else None)
            }
    return data


def powerset(s):
    """
    Return a list of all possible subsets of set s.
    """
    s = list(s)
    return [
        set(s) for s in itertools.chain.from_iterable(
            itertools.combinations(s, r) for r in range(len(s) + 1)
        )
    ]

def case_1(person,one_gene,two_genes,have_trait):
    if person in one_gene:
        if person in have_trait:
            return PROBS['gene'][1]*PROBS['trait'][1][True]
        else:
            return PROBS['gene'][1] * PROBS['trait'][1][False]
    elif person in two_genes:
        if person in have_trait:
            return PROBS['gene'][2]*PROBS['trait'][1][True]
        else:
            return PROBS['gene'][2] * PROBS['trait'][1][False]
    else:
        if person in have_trait:
            return PROBS['gene'][0]*PROBS['trait'][1][True]
        else:
            return PROBS['gene'][0] * PROBS['trait'][1][False]

def find_parent_genes(person,one_gene,two_genes):
    person_m = person['mother']
    person_f = person['father']
    genes=[]
    if person_m in one_gene:
        genes.append(1)
    elif person_m in two_genes:
        genes.append(2)
    else:
        genes.append(0)
    if person_f in one_gene:
        genes.append(1)
    elif person_f in two_genes:
        genes.append(2)
    else:
        genes.append(0)

    return genes

def get_probs_from_parent(genes,n):
    m=PROBS['mutation']
    if 1 not in genes and 2 not in genes:#[0,0]
        if n==0:
            return (1-m)*(1-m)
        if n==1:
            return ((1-m)*m)*2
        if n==2:
            return (m*m)

    if 1 in genes and 0 in genes and 2 not in genes:  # [0,1]
        if n == 0:
            return ((1-m)*m*0.5)+((1-m)*(1-m)*0.5)+(m*0.5*(1-m))+(m*0.5*m)
        if n == 1:
            return (m*m*0.5)+(m*(1-m)*0.5)+((1-m)*(1-m)*0.5)+((1-m)*m*0.5)
        if n == 2:
            return (m*(1-m)*0.5)+(m*m*0.5)

    if 1 in genes and 0 not in genes and 2 not in genes:  # [1,1]
        if n == 0:
            return 0.25*(((1-m)*(1-m))+((1-m)*(m))*2+((m)*(m)))
        if n == 1:
            return 0.25*((m*m)+(m*(1-m))+((1-m)*(1-m))*2)
        if n == 2:
            return 0.25*((m*m)+(m*(1-m))*2+((1-m)*(1-m)))

    if 1 in genes and 0 not in genes and 2 in genes:  # [1,2]
        if n == 0:
            return 0.25*(2*(m*(1-m))+2*(m*m))
        if n == 1:
            return 0.25*(2*((1-m)*(1-m))+4*(m*(1-m)))
        if n == 2:
            return 0.25*(2*(m*(1-m))+2*((1-m)*(1-m)))

    if 1 not in genes and 0 not in genes and 2 in genes:  # [2,2]
        if n == 0:
            return m*m
        if n == 1:
            return 2*(m*(1-m))
        if n == 2:
            return (1-m)*(1-m)

    if 1 not in genes and 0 not in genes and 2 in genes:  # [0,2]
        if n == 0:
            return m*(1-m)
        if n == 1:
            return m*(1-m)
        if n == 2:
            return (m)*(1-m)
def case_2(p,one_gene,two_genes,have_trait):
    person=p['name']
    genes=find_parent_genes(p,one_gene,two_genes)
    p1=None
    p2=None

    if person in one_gene:
        p2=get_probs_from_parent(genes,1)
        if person in have_trait:
            p1=PROBS['trait'][1][True]
        else:
            p1=PROBS['trait'][1][False]
        return p2*p1
    elif person in two_genes:
        p2 = get_probs_from_parent(genes, 2)
        if person in have_trait:
            p1=PROBS['trait'][1][True]
        else:
            p1=PROBS['trait'][1][False]
        return p2 * p1
    else:
        p2 = get_probs_from_parent(genes, 0)
        if person in have_trait:
            p1=PROBS['trait'][1][True]
        else:
            p1=PROBS['trait'][1][False]
        return p2 * p1

def joint_probability(people, one_gene, two_genes, have_trait):
    """
    Compute and return a joint probability.

    The probability returned should be the probability that
        * everyone in set `one_gene` has one copy of the gene, and
        * everyone in set `two_genes` has two copies of the gene, and
        * everyone not in `one_gene` or `two_gene` does not have the gene, and
        * everyone in set `have_trait` has the trait, and
        * everyone not in set` have_trait` does not have the trait.
    """
    p=1
    for i in people:
        person_m=i['mother']
        person_f=i['father']
        if person_m==None or person_f==None:
            p*=case_1(i['name'],one_gene, two_genes, have_trait)
        else:
            p*case_2(i,one_gene, two_genes, have_trait)
    return p

def update(probabilities, one_gene, two_genes, have_trait, p):
    """
    Add to `probabilities` a new joint probability `p`.
    Each person should have their "gene" and "trait" distributions updated.
    Which value for each distribution is updated depends on whether
    the person is in `have_gene` and `have_trait`, respectively.
    """
    for i in probabilities:
        if i in one_gene:
            probabilities[i]['gene'][1]=p
        elif i in two_genes:
            probabilities[i]['gene'][2]=p
        else:
            probabilities[i]['gene'][0]=p

        if i in have_trait:
            probabilities[i]['trait'][True]=p
        else:
            probabilities[i]['trait'][False]=p

def normalize(probabilities):
    """
    Update `probabilities` such that each probability distribution
    is normalized (i.e., sums to 1, with relative proportions the same).
    """
    s=0
    if sum(probabilities.values())!=1:
        s=sum(probabilities.values())

    for i in probabilities.keys():
        probabilities[i]/=s


if __name__ == "__main__":
    main()


# crossword- generate.py

In [ ]:
import math
import sys

from crossword import *


class CrosswordCreator():

    def __init__(self, crossword):
        """
        Create new CSP project3 generate.
        """
        self.crossword = crossword
        self.domains = {
            var: self.crossword.words.copy()
            for var in self.crossword.variables
        }

    def letter_grid(self, assignment):
        """
        Return 2D array representing a given assignment.
        """
        letters = [
            [None for _ in range(self.crossword.width)]
            for _ in range(self.crossword.height)
        ]
        for variable, word in assignment.items():
            direction = variable.direction
            for k in range(len(word)):
                i = variable.i + (k if direction == Variable.DOWN else 0)
                j = variable.j + (k if direction == Variable.ACROSS else 0)
                letters[i][j] = word[k]
        return letters

    def print(self, assignment):
        """
        Print project3 assignment to the terminal.
        """
        letters = self.letter_grid(assignment)
        for i in range(self.crossword.height):
            for j in range(self.crossword.width):
                if self.crossword.structure[i][j]:
                    print(letters[i][j] or " ", end="")
                else:
                    print("█", end="")
            print()

    def save(self, assignment, filename):
        """
        Save project3 assignment to an image file.
        """
        from PIL import Image, ImageDraw, ImageFont
        cell_size = 100
        cell_border = 2
        interior_size = cell_size - 2 * cell_border
        letters = self.letter_grid(assignment)

        # Create a blank canvas
        img = Image.new(
            "RGBA",
            (self.crossword.width * cell_size,
             self.crossword.height * cell_size),
            "black"
        )
        font = ImageFont.truetype("assets/fonts/OpenSans-Regular.ttf", 80)
        draw = ImageDraw.Draw(img)

        for i in range(self.crossword.height):
            for j in range(self.crossword.width):

                rect = [
                    (j * cell_size + cell_border,
                     i * cell_size + cell_border),
                    ((j + 1) * cell_size - cell_border,
                     (i + 1) * cell_size - cell_border)
                ]
                if self.crossword.structure[i][j]:
                    draw.rectangle(rect, fill="white")
                    if letters[i][j]:
                        _, _, w, h = draw.textbbox((0, 0), letters[i][j], font=font)
                        draw.text(
                            (rect[0][0] + ((interior_size - w) / 2),
                             rect[0][1] + ((interior_size - h) / 2) - 10),
                            letters[i][j], fill="black", font=font
                        )

        img.save(filename)

    def solve(self):
        """
        Enforce node and arc consistency, and then solve the CSP.
        """
        self.enforce_node_consistency()
        self.ac3()
        return self.backtrack(dict())

    def enforce_node_consistency(self):
        """
        Update `self.domains` such that each variable is node-consistent.
        (Remove any values that are inconsistent with a variable's unary
         constraints; in this case, the length of the word.)
        """
        for var in crossword.variables:
            for i in self.domains[var]:
                if len(i)!=var.length:
                    self.domains[var].remove(i)

    def revise(self, x, y):
        """
        Make variable `x` arc consistent with variable `y`.
        To do so, remove values from `self.domains[x]` for which there is no
        possible corresponding value for `y` in `self.domains[y]`.

        Return True if a revision was made to the domain of `x`; return
        False if no revision was made.
        """
        revised=False
        for a in self.domains[x]:
            for b in self.domains[y]:
                if (a,b) in self.crossword.overlaps:
                    self.domains[x].remove(a)
                    revised=True
            return revised

    def ac3(self, arcs=None):
        """
        Update `self.domains` such that each variable is arc consistent.
        If `arcs` is None, begin with initial list of all arcs in the problem.
        Otherwise, use `arcs` as the initial list of arcs to make consistent.

        Return True if arc consistency is enforced and no domains are empty;
        return False if one or more domains end up empty.
        """
        if arcs==None:
            for i in range(len(crossword.variables)):
                for j in range(i+1,-len(crossword.variables)):
                    token=(crossword.variables[i],crossword.variables[j])
                    arcs.append(token)

        while len(arcs)!=0:
            arc=arcs[0]
            arcs=arcs[1:]
            result=self.revise(arc[0],arc[1])

            if result:
                if len(self.domains[arc[0]])==0:
                    return False
                for i in crossword.neighbors(arc[0]):
                    tok=None
                    if i!= arc[1]:
                        tok=(arc[0],i)
                        arcs.append(tok)
            return True


    def assignment_complete(self, assignment):
        """
        Return True if `assignment` is complete (i.e., assigns a value to each
        project3 variable); return False otherwise.
        """
        for i in crossword.variables:
            if len(assignment[i])==0:
                return False
        return True

    def check(self,x,y,overlaps,assignment):
        a=assignment[x]
        b=assignment[y]
        if x.direction=='down':
            if y.direction=='down':
                for o in overlaps:
                    if a[o[0]]!=b[o[0]]:
                        return False

        if x.direction=='down':
            if y.direction=='across':
                for o in overlaps:
                    if a[o[0]]!=b[o[1]]:
                        return False

        if x.direction=='across':
            if y.direction=='down':
                for o in overlaps:
                    if a[o[1]]!=b[o[0]]:
                        return False

        if x.direction=='across':
            if y.direction=='across':
                for o in overlaps:
                    if a[o[1]]!=b[o[1]]:
                        return False
        return True

    def consistent(self, assignment):
        """
        Return True if `assignment` is consistent (i.e., words fit in project3
        puzzle without conflicting characters); return False otherwise.
        """
        #check unigueness
        words_used=set()
        for i in crossword.variables:
            if assignment[i] in words_used:
                return False
            else:
                words_used.add(assignment[i])

        #check length
        for i in crossword.variables:
            if len(assignment[i])!=i.length:
                return False

        #check overlaps
        flag=None
        for i in range(len(crossword.variables)):
            for j in range(i+1,len(crossword.variables)):
                overlaps=set()
                for k in crossword.variables[i].cells and k in crossword.variables[j].cells:
                    overlaps.add(k)
                if len(overlaps)!=0:
                    flag=self.check(crossword.variables[i],crossword.variables[j],overlaps,assignment)
                if not flag:
                    return False
        if not flag:
            return False
        return True



    def order_domain_values(self, var, assignment):
        """
        Return a list of values in the domain of `var`, in order by
        the number of values they rule out for neighboring variables.
        The first value in the list, for example, should be the one
        that rules out the fewest values among the neighbors of `var`.
        """
        words=set(self.domains[var])
        n=crossword.neighbors(var)
        dicto=dict()
        for i in n:
            words_n=set(self.domains[i])
            intr=words.intersection(words_n)
            dicto[i]=len(intr)

        m=-math.inf
        mm=None
        for p,q in dicto.items():
            if q>m or mm==None:
                m=q
                mm=p

        return mm

    def number_of_values_in_domain(self, var):
        return len(self.domains[var])

    def select_unassigned_variable(self, assignment):
        """
        Return an unassigned variable not already part of `assignment`.
        Choose the variable with the minimum number of remaining values
        in its domain. If there is a tie, choose the variable with the highest
        degree. If there is a tie, any of the tied variables are acceptable
        return values.
        """
        all_variables=set(self.domains.keys())
        assigned_variables=set(assignment.keys())
        unassigned_variables=list(all_variables.difference(assigned_variables)).sort(key=self.number_of_values_in_domain)
        return unassigned_variables

    def backtrack(self, assignment):
        """
        Using Backtracking Search, take as input a partial assignment for the
        project3 and return a complete assignment if possible to do so.

        `assignment` is a mapping from variables (keys) to words (values).

        If no assignment is possible, return None.
        """
        if self.assignment_complete(assignment):
            return assignment
        var=self.select_unassigned_variable(assignment)
        result=None
        for value in self.domains[var]:
            assignment[var]=value
            result=self.backtrack(assignment)
        if result!=False:
            return result
        assignment.remove(var)


def main():

    # Check usage
    if len(sys.argv) not in [3, 4]:
        sys.exit("Usage: python generate.py structure words [output]")

    # Parse command-line arguments
    structure = sys.argv[1]
    words = sys.argv[2]
    output = sys.argv[3] if len(sys.argv) == 4 else None

    # Generate project3
    crossword = Crossword(structure, words)
    creator = CrosswordCreator(crossword)
    assignment = creator.solve()

    # Print result
    if assignment is None:
        print("No solution.")
    else:
        creator.print(assignment)
        if output:
            creator.save(assignment, output)


if __name__ == "__main__":
    main()


# shop

In [ ]:
import csv
import sys
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

TEST_SIZE = 0.4


def main():

    # Check command-line arguments
    if len(sys.argv) != 2:
        sys.exit("Usage: python s.py data")

    # Load data from spreadsheet and split into train and test sets
    evidence, labels = load_data(sys.argv[1])
    X_train, X_test, y_train, y_test = train_test_split(
        evidence, labels, test_size=TEST_SIZE
    )

    # Train model and make predictions
    model = train_model(X_train, y_train)
    predictions = model.predict(X_test)
    sensitivity, specificity = evaluate(y_test, predictions)

    # Print results
    print(f"Correct: {(y_test == predictions).sum()}")
    print(f"Incorrect: {(y_test != predictions).sum()}")
    print(f"True Positive Rate: {100 * sensitivity:.2f}%")
    print(f"True Negative Rate: {100 * specificity:.2f}%")

MONTHS={
    'Jan':0,
    'Feb':1,
    'Mar':2,
    'Apr':3,
    'May':4,
    'Jun':5,
    'July':6,
    'Aug':7,
    'Sep':8,
    'Oct':9,
    'Nov':10,
    'Dec':11
}

VIS_TYPE={
    'Returning':1,
    'New_Visitor':0
}


class helper_functions():
    def give_month_number(self,st):
        return MONTHS[st]

    def visitor_type(self,st):
        return VIS_TYPE[st]

    def weekend_map(self,st):
        if st=='TRUE':
            return 1
        else:
            return 0

    def revenue_map(self,st):
        if st=='TRUE':
            return 1
        else:
            return 0

def load_data(filename):
    """
    Load shopping data from a CSV file `filename` and convert into a list of
    evidence lists and a list of labels. Return a tuple (evidence, labels).

    evidence should be a list of lists, where each list contains the
    following values, in order:
        - Administrative, an integer
        - Administrative_Duration, a floating point number
        - Informational, an integer
        - Informational_Duration, a floating point number
        - ProductRelated, an integer
        - ProductRelated_Duration, a floating point number
        - BounceRates, a floating point number
        - ExitRates, a floating point number
        - PageValues, a floating point number
        - SpecialDay, a floating point number
        - Month, an index from 0 (January) to 11 (December)
        - OperatingSystems, an integer
        - Browser, an integer
        - Region, an integer
        - TrafficType, an integer
        - VisitorType, an integer 0 (not returning) or 1 (returning)
        - Weekend, an integer 0 (if false) or 1 (if true)

    labels should be the corresponding list of labels, where each label
    is 1 if Revenue is true, and 0 otherwise.
    """
    f=pd.read_csv(filename)

    data={
        'evidence':[],
        'labels':[]
    }
    f=f.astype({
        'Administrative':int,
        'Informational':int,
        'ProductRelated':int,
        'Month':str,
        'OperatingSystems':int,
        'Browser':int,
        'Region':int,
        'TrafficType':int,
        'VisitorType':str,
        'Weekend':str,
        'Administrative_Duration':float,
        'Informational_Duration':float,
        'ProductRelated_Duration':float,
        'BounceRates':float,
        'ExitRates':float,
        'PageValues':float,
        'SpecialDay':float,
        'Revenue':str
    })
    f['Month'] = f['Month'].apply(lambda x: helper_functions.give_month_number(x))
    f['VisitorType'] = f['VisitorType'].apply(lambda x: helper_functions.visitor_type(x))
    f['Weekend'] = f['Weekend'].apply(lambda x: helper_functions.weekend_map(x))
    f['Revenue'] = f['Revenue'].apply(lambda x: helper_functions.revenue_map(x))

    evidence=f[['Administrative',
        'Informational',
        'ProductRelated',
        'Month',
        'OperatingSystems',
        'Browser',
        'Region',
        'TrafficType',
        'VisitorType',
        'Weekend',
        'Administrative_Duration',
        'Informational_Duration',
        'ProductRelated_Duration',
        'BounceRates',
        'ExitRates',
        'PageValues',
        'SpecialDay',]]


    labels=f[['Revenue']]

    for index, rows in evidence.iterrows():
        data['evidence'].append([rows.Administrative, rows.Administrative_Duration,
                                 rows.Informational, rows.Informational_Duration,
                                 rows.ProductRelated, rows.ProductRelated_Duration,
                                 rows.BounceRates, rows.ExitRates, rows.PageValues, rows.SpecialDay,
                                 rows.B.Month, rows.OperatingSystems, rows.Browser, rows.Region,
                                 rows.TrafficType,rows.VisitorType, rows.Weekend,
                                 ])

    for index, rows in labels.iterrows():
        data['labels'].append([rows.Revenue])

    return tuple(data['evidence'],data['labels'])

def train_model(evidence, labels):
    """
    Given a list of evidence lists and a list of labels, return a
    fitted k-nearest neighbor model (k=1) trained on the data.
    """
    model=KNeighborsClassifier(n_neighbors=1)
    model.fit(evidence,labels)

    return model

def evaluate(labels, predictions):
    """
    Given a list of actual labels and a list of predicted labels,
    return a tuple (sensitivity, specificity).

    Assume each label is either a 1 (positive) or 0 (negative).

    `sensitivity` should be a floating-point value from 0 to 1
    representing the "true positive rate": the proportion of
    actual positive labels that were accurately identified.

    `specificity` should be a floating-point value from 0 to 1
    representing the "true negative rate": the proportion of
    actual negative labels that were accurately identified.
    """

    for i in range(predictions):
        sen=0
        spe=0
        if labels[i]==1:
            if predictions[i]==1:
                sen+=1
        if labels[i]==0:
            if predictions[i]==0:
                spe+=1

        sen/=len(labels)
        spe/=len(labels)

        return (sen,spe)

if __name__ == "__main__":
    main()


# nim

In [ ]:
import math
import random
import time


class Nim():

    def __init__(self, initial=[1, 3, 5, 7]):
        """
        Initialize game board.
        Each game board has
            - `piles`: a list of how many elements remain in each pile
            - `player`: 0 or 1 to indicate which player's turn
            - `winner`: None, 0, or 1 to indicate who the winner is
        """
        self.piles = initial.copy()
        self.player = 0
        self.winner = None

    @classmethod
    def available_actions(cls, piles):
        """
        Nim.available_actions(piles) takes a `piles` list as input
        and returns all of the available actions `(i, j)` in that state.

        Action `(i, j)` represents the action of removing `j` items
        from pile `i` (where piles are 0-indexed).
        """
        actions = set()
        for i, pile in enumerate(piles):
            for j in range(1, pile + 1):
                actions.add((i, j))
        return actions

    @classmethod
    def other_player(cls, player):
        """
        Nim.other_player(player) returns the player that is not
        `player`. Assumes `player` is either 0 or 1.
        """
        return 0 if player == 1 else 1

    def switch_player(self):
        """
        Switch the current player to the other player.
        """
        self.player = Nim.other_player(self.player)

    def move(self, action):
        """
        Make the move `action` for the current player.
        `action` must be a tuple `(i, j)`.
        """
        pile, count = action

        # Check for errors
        if self.winner is not None:
            raise Exception("Game already won")
        elif pile < 0 or pile >= len(self.piles):
            raise Exception("Invalid pile")
        elif count < 1 or count > self.piles[pile]:
            raise Exception("Invalid number of objects")

        # Update pile
        self.piles[pile] -= count
        self.switch_player()

        # Check for a winner
        if all(pile == 0 for pile in self.piles):
            self.winner = self.player


class NimAI():

    def __init__(self, alpha=0.5, epsilon=0.1):
        """
        Initialize AI with an empty Q-learning dictionary,
        an alpha (learning) rate, and an epsilon rate.

        The Q-learning dictionary maps `(state, action)`
        pairs to a Q-value (a number).
         - `state` is a tuple of remaining piles, e.g. (1, 1, 4, 4)
         - `action` is a tuple `(i, j)` for an action
        """
        self.q = dict()
        self.alpha = alpha
        self.epsilon = epsilon

    def update(self, old_state, action, new_state, reward):
        """
        Update Q-learning model, given an old state, an action taken
        in that state, a new resulting state, and the reward received
        from taking that action.
        """
        old = self.get_q_value(old_state, action)
        best_future = self.best_future_reward(new_state)
        self.update_q_value(old_state, action, old, reward, best_future)

    def get_q_value(self, state, action):
        """
        Return the Q-value for the state `state` and the action `action`.
        If no Q-value exists yet in `self.q`, return 0.
        """
        if self.q[(tuple(state),action)]:
            return self.q[(tuple(state),action)]
        else:
            return 0

    def update_q_value(self, state, action, old_q, reward, future_rewards):
        """
        Update the Q-value for the state `state` and the action `action`
        given the previous Q-value `old_q`, a current reward `reward`,
        and an estiamte of future rewards `future_rewards`.

        Use the formula:

        Q(s, a) <- old value estimate
                   + alpha * (new value estimate - old value estimate)

        where `old value estimate` is the previous Q-value,
        `alpha` is the learning rate, and `new value estimate`
        is the sum of the current reward and estimated future rewards.
        """

        new_q=old_q+self.alpha(reward+future_rewards-old_q)
        self.update(state,action,new_q)

    def best_future_reward(self, state):
        """
        Given a state `state`, consider all possible `(state, action)`
        pairs available in that state and return the maximum of all
        of their Q-values.

        Use 0 as the Q-value if a `(state, action)` pair has no
        Q-value in `self.q`. If there are no available actions in
        `state`, return 0.
        """
        best=None
        act=None
        for a in Nim.available_actions(state):
            if best==None or self.get_q_value(state,a)>best:
                best=self.get_q_value(state,a)
                act=a
        if best==None:
            return 0

        return best

    def choose_action(self, state, epsilon=True):
        """
        Given a state `state`, return an action `(i, j)` to take.

        If `epsilon` is `False`, then return the best action
        available in the state (the one with the highest Q-value,
        using 0 for pairs that have no Q-values).

        If `epsilon` is `True`, then with probability
        `self.epsilon` choose a random available action,
        otherwise choose the best action available.

        If multiple actions have the same Q-value, any of those
        options is an acceptable return value.
        """

        best = None
        act = None
        for a in Nim.available_actions(state):
            if best == None or self.get_q_value(state, a) > best:
                best = self.get_q_value(state, a)
                act = a

        if not self.epsilon:
            if best==None:
                return 0
            else:
                return act

        else:
            return random.choices(Nim.available_actions(state), k=1)[0]



def train(n):
    """
    Train an AI by playing `n` games against itself.
    """

    player = NimAI()

    # Play n games
    for i in range(n):
        print(f"Playing training game {i + 1}")
        game = Nim()

        # Keep track of last move made by either player
        last = {
            0: {"state": None, "action": None},
            1: {"state": None, "action": None}
        }

        # Game loop
        while True:

            # Keep track of current state and action
            state = game.piles.copy()
            action = player.choose_action(game.piles)

            # Keep track of last state and action
            last[game.player]["state"] = state
            last[game.player]["action"] = action

            # Make move
            game.move(action)
            new_state = game.piles.copy()

            # When game is over, update Q values with rewards
            if game.winner is not None:
                player.update(state, action, new_state, -1)
                player.update(
                    last[game.player]["state"],
                    last[game.player]["action"],
                    new_state,
                    1
                )
                break

            # If game is continuing, no rewards yet
            elif last[game.player]["state"] is not None:
                player.update(
                    last[game.player]["state"],
                    last[game.player]["action"],
                    new_state,
                    0
                )

    print("Done training")

    # Return the trained AI
    return player


def play(ai, human_player=None):
    """
    Play human game against the AI.
    `human_player` can be set to 0 or 1 to specify whether
    human player moves first or second.
    """

    # If no player order set, choose human's order randomly
    if human_player is None:
        human_player = random.randint(0, 1)

    # Create new game
    game = Nim()

    # Game loop
    while True:

        # Print contents of piles
        print()
        print("Piles:")
        for i, pile in enumerate(game.piles):
            print(f"Pile {i}: {pile}")
        print()

        # Compute available actions
        available_actions = Nim.available_actions(game.piles)
        time.sleep(1)

        # Let human make a move
        if game.player == human_player:
            print("Your Turn")
            while True:
                pile = int(input("Choose Pile: "))
                count = int(input("Choose Count: "))
                if (pile, count) in available_actions:
                    break
                print("Invalid move, try again.")

        # Have AI make a move
        else:
            print("AI's Turn")
            pile, count = ai.choose_action(game.piles, epsilon=False)
            print(f"AI chose to take {count} from pile {pile}.")

        # Make move
        game.move((pile, count))

        # Check for winner
        if game.winner is not None:
            print()
            print("GAME OVER")
            winner = "Human" if game.winner == human_player else "AI"
            print(f"Winner is {winner}")
            return


# traffic

In [ ]:
import cv2
import numpy as np
import os
import sys
import tensorflow as tf

from sklearn.model_selection import train_test_split

EPOCHS = 10
IMG_WIDTH = 30
IMG_HEIGHT = 30
NUM_CATEGORIES = 43
TEST_SIZE = 0.4


def main():

    # Check command-line arguments
    if len(sys.argv) not in [2, 3]:
        sys.exit("Usage: python traffic.py data_directory [model.h5]")

    # Get image arrays and labels for all image files
    images, labels = load_data(sys.argv[1])

    # Split data into training and testing sets
    labels = tf.keras.utils.to_categorical(labels)
    x_train, x_test, y_train, y_test = train_test_split(
        np.array(images), np.array(labels), test_size=TEST_SIZE
    )

    # Get a compiled neural network
    model = get_model()

    # Fit model on training data
    model.fit(x_train, y_train, epochs=EPOCHS)

    # Evaluate neural network performance
    model.evaluate(x_test,  y_test, verbose=2)

    # Save model to file
    if len(sys.argv) == 3:
        filename = sys.argv[2]
        model.save(filename)
        print(f"Model saved to {filename}.")


def load_data(data_dir):
    """
    Load image data from directory `data_dir`.

    Assume `data_dir` has one directory named after each category, numbered
    0 through NUM_CATEGORIES - 1. Inside each category directory will be some
    number of image files.

    Return tuple `(images, labels)`. `images` should be a list of all
    of the images in the data directory, where each image is formatted as a
    numpy ndarray with dimensions IMG_WIDTH x IMG_HEIGHT x 3. `labels` should
    be a list of integer labels, representing the categories for each of the
    corresponding `images`.
    """
    list_of_images=[]
    list_of_labels=[]
    list_of_dir=os.listdir(data_dir)
    l=None
    for l in list_of_dir:
        new_dir=os.path.join(data_dir,l)
        list_of_images=os.listdir(new_dir)
        for i in list_of_images:
            img = cv.imread(i)
            res = cv.resize(img, None, fx=IMG_WIDTH, fy=IMG_HEIGHT, interpolation=cv.INTER_CUBIC)
            list_of_images.append(res)
            list_of_labels.append(l)

    return (list_of_images,list_of_labels)



def get_model():
    """
    Returns a compiled convolutional neural network model. Assume that the
    `input_shape` of the first layer is `(IMG_WIDTH, IMG_HEIGHT, 3)`.
    The output layer should have `NUM_CATEGORIES` units, one for each category.
    """
    model=tf.keras.Sequential([
        # Convolutional layer. Learn 32 filters using a 3x3 kernel
        tf.keras.layers.Conv2D(
            32, (3, 3), activation="relu", input_shape=(IMG_WIDTH, IMG_HEIGHT, 3)
        ),

        # Max-pooling layer, using 2x2 pool size
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

        # Convolutional layer. Learn 32 filters using a 3x3 kernel
        tf.keras.layers.Conv2D(
            16, (3, 3), activation="relu"
        ),

        # Max-pooling layer, using 2x2 pool size
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

        # Flatten units
        tf.keras.layers.Flatten(),

        # Add a hidden layer with dropout
        tf.keras.layers.Dense(NUM_CATEGORIES, activation="relu"),
        tf.keras.layers.Dropout(0.5),

        # Add an output layer with output units for all 10 digits
        tf.keras.layers.Dense(10, activation="softmax")
    ])
    model.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

if __name__ == "__main__":
    main()


# parser

In [ ]:
import nltk
import sys

TERMINALS = """
Adj -> "country" | "dreadful" | "enigmatical" | "little" | "moist" | "red"
Adv -> "down" | "here" | "never"
Conj -> "and" | "until"
Det -> "a" | "an" | "his" | "my" | "the"
N -> "armchair" | "companion" | "day" | "door" | "hand" | "he" | "himself"
N -> "holmes" | "home" | "i" | "mess" | "paint" | "palm" | "pipe" | "she"
N -> "smile" | "thursday" | "walk" | "we" | "word"
P -> "at" | "before" | "in" | "of" | "on" | "to"
V -> "arrived" | "came" | "chuckled" | "had" | "lit" | "said" | "sat"
V -> "smiled" | "tell" | "were"
"""

NONTERMINALS = """
S -> N V | S Conj S
N-> A | Adj N | B | N Conj N | A B
A-> Det N
B-> P N | N P N
V-> V N | N Conj V | V Conj V | X | Y |
X -> V Adv
Y -> Adv V
"""

grammar = nltk.CFG.fromstring(NONTERMINALS + TERMINALS)
parser = nltk.ChartParser(grammar)


def main():

    # If filename specified, read sentence from file
    if len(sys.argv) == 2:
        with open(sys.argv[1]) as f:
            s = f.read()

    # Otherwise, get sentence as input
    else:
        s = input("Sentence: ")

    # Convert input into list of words
    s = preprocess(s)

    # Attempt to parse sentence
    try:
        trees = list(parser.parse(s))
    except ValueError as e:
        print(e)
        return
    if not trees:
        print("Could not parse sentence.")
        return

    # Print each tree with noun phrase chunks
    for tree in trees:
        tree.pretty_print()

        print("Noun Phrase Chunks")
        for np in np_chunk(tree):
            print(" ".join(np.flatten()))


def preprocess(sentence):
    """
    Convert `sentence` to a list of its words.
    Pre-process sentence by converting all characters to lowercase
    and removing any word that does not contain at least one alphabetic
    character.
    """

    words=sentence.split(sep=' ')
    words_cleaned=[]
    for i in words:
        i.lower()
        if i.isalpha():
            words_cleaned.append(i)
    return words_cleaned

def np_chunk(tree):
    """
    Return a list of all noun phrase chunks in the sentence tree.
    A noun phrase chunk is defined as any subtree of the sentence
    whose label is "NP" that does not itself contain any other
    noun phrases as subtrees.
    """
    t=tree
    while(len(t)!=0):
        flag=0
        if t.label()=='NP':
            for j in t.leaves:
                if j.label()=='NP':
                    flag=1
                    break
        if flag==0:
            return t
        t=t.leaves()[0]


if __name__ == "__main__":
    main()


# questions

In [ ]:
import copy
import math
import os

import nltk
import sys

from nltk.tokenize import sent_tokenize, word_tokenize
import string

FILE_MATCHES = 1
SENTENCE_MATCHES = 1


def main():

    # Check command-line arguments
    if len(sys.argv) != 2:
        sys.exit("Usage: python questions.py corpus")

    # Calculate IDF values across files
    files = load_files(sys.argv[1])
    file_words = {
        filename: tokenize(files[filename])
        for filename in files
    }
    file_idfs = compute_idfs(file_words)

    # Prompt user for query
    query = set(tokenize(input("Query: ")))

    # Determine top file matches according to TF-IDF
    filenames = top_files(query, file_words, file_idfs, n=FILE_MATCHES)

    # Extract sentences from top files
    sentences = dict()
    for filename in filenames:
        for passage in files[filename].split("\n"):
            for sentence in nltk.sent_tokenize(passage):
                tokens = tokenize(sentence)
                if tokens:
                    sentences[sentence] = tokens

    # Compute IDF values across sentences
    idfs = compute_idfs(sentences)

    # Determine top sentence matches
    matches = top_sentences(query, sentences, idfs, n=SENTENCE_MATCHES)
    for match in matches:
        print(match)


def load_files(directory):
    """
    Given a directory name, return a dictionary mapping the filename of each
    `.txt` file inside that directory to the file's contents as a string.
    """
    list_of_dir=os.listdir(directory)
    dict_of_docs=dict()
    for i in list_of_dir:
        file=os.path.join(directory,i)
        f=open(file)
        dict_of_docs[i]=os.read(f)
        f.close()



def tokenize(document):
    """
    Given a document (represented as a string), return a list of all of the
    words in that document, in order.

    Process document by coverting all words to lowercase, and removing any
    punctuation or English stopwords.
    """

    l=list(word_tokenize(document))
    for i in range(len(l)):
        l[i]=l[i].lower()
        if l[i] in string.punctuation or l[i] in nltk.corpus.stopwords.words("english"):
            l.pop(i)


def compute_idfs(documents):
    """
    Given a dictionary of `documents` that maps names of documents to a list
    of words, return a dictionary that maps words to their IDF values.

    Any word that appears in at least one of the documents should be in the
    resulting dictionary.
    """
    doc_set_copy=copy.deepcopy(documents)
    all_words=dict()
    words = set()
    for i in doc_set_copy.keys():
        doc_set_copy[i]=set(doc_set_copy[i])
        words.union(doc_set_copy[i])

    for i in words:
        count=0
        for j in doc_set_copy.keys():
            if i in doc_set_copy[j]:
                count=doc_set_copy[j].count(i)

        all_words[i]=count

    return all_words


def top_files(query, files, idfs, n):
    """
    Given a `query` (a set of words), `files` (a dictionary mapping names of
    files to a list of their words), and `idfs` (a dictionary mapping words
    to their IDF values), return a list of the filenames of the the `n` top
    files that match the query, ranked according to tf-idf.
    """
    best_files_dict=dict()

    for j in files.keys():
        best_files_dict[j]=dict()
        for i in query:
            best_files_dict[j][i]=files[j].count(i)*idfs(i)

    best_files=dict()
    for j in files.keys():
        best_files[j] = 0
        for i in query:
            best_files[j]+=best_files_dict[j][i]

    ans=sorted(best_files.items(), key=lambda kv:
    (kv[1], kv[0]))

    return ans[:n]


def top_sentences(query, sentences, idfs, n):
    """
    Given a `query` (a set of words), `sentences` (a dictionary mapping
    sentences to a list of their words), and `idfs` (a dictionary mapping words
    to their IDF values), return a list of the `n` top sentences that match
    the query, ranked according to idf. If there are ties, preference should
    be given to sentences that have a higher query term density.
    """
    dicto=dict()
    for i in sentences:
        d=dict()
        for j in query:
            c=i.count(j)
            dicto[sentences.index(i)][j]=c*idfs(j)

    ans = sorted(dicto.items(), key=lambda kv:
    (kv[1], kv[0]))

    return ans[:n]


if __name__ == "__main__":
    main()
